In [1]:
from mindspore import context
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceField
from mindsponge import DynamicUpdater
from mindsponge.control import VelocityVerlet, Langevin
from mindsponge.callback import WriteH5MD, RunInfo

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

## 使用力场模板

MindSponge中支持yaml格式的力场参数信息配置，其中大体分为`模板`和`参数`两个大类。`模板`中主要存储系统中的原子名称、原子类型、原子质量、原子序数、成键关系等基本信息，也是一般都不需要去改动的信息。`参数`中顾名思义就是存储一些力场参数，比如键长相互作用$E_b=k_b(l-l_0)^2$中的$(k_b,l_0)$就是一组力场参数。不同的原子组合形成的键、角度、二面角还有其他的一些远程相互作用等，都会对应于不同的参数，在不同的力场中千差万别。

比如这是一个三点水的模板配置信息：

```yaml
template:
  WAT:
    atom_name: [O, H1, H2]
    atom_type: [OW, HW, HW]
    atom_mass: [16.00, 1.008, 1.008]
    atomic_number: [8, 1, 1]
    bond:
    - [0, 1]
    - [0, 2]
    head_atom: null
    tail_atom: null
```

以及一个三点水的力场参数信息：

```yaml
template:
  base: water.tip3p.yaml
parameters:
  bond_energy:
    length_unit: nm
    energy_unit: kj/mol
    parameter_names:
      atoms: [bond_length, force_constant]
    parameters:
      OW-HW: [0.09572, 502416]
  angle_energy:
    length_unit: nm
    energy_unit: kj/mol
    parameter_names:
      atoms: [bond_angle, force_constant]
    parameters:
      HW-OW-HW: [1.824218, 628.02]
  coulomb_energy:
    length_unit: nm
    energy_unit: kj/mol
  vdw_energy:
    length_unit: nm
    energy_unit: kj/mol
    parameter_names:
      atoms: [sigma, epsilon]
    parameters:
      OW: [0.315061, 0.636386]
      HW: [0.0, 0.0]
```

用户可以根据不同的力场自定义一个这样的yaml文件，然后就可以在MindSponge平台上非常方便的进行分子动力学模拟。

In [3]:
system = Molecule(template='water.tip3p.yaml')

In [4]:
potential = ForceField(system, parameters='TIP3P')

In [5]:
opt = DynamicUpdater(system,
                     integrator=VelocityVerlet(system),
                     thermostat=Langevin(system, 300),
                     time_step=1e-3)

In [6]:
md = Sponge(system, potential, opt)

TotalTime = 0.392683, [19]
[parse]: 0.0140133
[symbol_resolve]: 0.0737499, [1]
    [Cycle 1]: 0.0730414, [1]
        [resolve]: 0.0730308
[combine_like_graphs]: 2.21469e-06
[inference_opt_prepare]: 0.000437286
[abstract_specialize]: 0.268016
[auto_monad]: 0.000853831
[inline]: 7.67224e-06
[py_pre_ad]: 1.1567e-06
[pynative_shard]: 3.26522e-06
[pipeline_split]: 5.23962e-06
[optimize]: 0.025488, [15]
    [simplify_data_structures]: 0.000386514
    [opt_a]: 0.0240513, [2]
        [Cycle 1]: 0.0228649, [25]
            [expand_dump_flag]: 1.41133e-05
            [switch_simplify]: 0.000805767
            [a_1]: 0.00816911
            [recompute_prepare]: 2.27783e-05
            [updatestate_depend_eliminate]: 2.56188e-05
            [updatestate_assign_eliminate]: 2.19308e-05
            [updatestate_loads_eliminate]: 2.1752e-05
            [parameter_eliminate]: 3.9693e-06
            [a_2]: 0.00086844
            [accelerated_algorithm]: 1.7738e-05
            [auto_parallel]: 4.64171e-06

In [7]:
run_info = RunInfo(10)
cb_h5md = WriteH5MD(system, 'tutorial_b02.h5md', save_freq=10, write_velocity=True, write_force=True)

In [8]:
md.run(1000, callbacks=[run_info, cb_h5md])

TotalTime = 0.0119108, [19]
[parse]: 0.0033556
[symbol_resolve]: 0.00103726, [1]
    [Cycle 1]: 0.00100756, [1]
        [resolve]: 0.00100202
[combine_like_graphs]: 5.4203e-07
[inference_opt_prepare]: 2.74163e-05
[abstract_specialize]: 0.000581639
[auto_monad]: 0.000116115
[inline]: 1.31689e-06
[py_pre_ad]: 1.24425e-06
[pynative_shard]: 1.0822e-06
[pipeline_split]: 1.86265e-06
[optimize]: 0.00180757, [15]
    [simplify_data_structures]: 7.46921e-06
    [opt_a]: 0.00146777, [2]
        [Cycle 1]: 0.000845667, [25]
            [expand_dump_flag]: 1.30758e-06
            [switch_simplify]: 9.89996e-06
            [a_1]: 0.000163199
            [recompute_prepare]: 3.64147e-06
            [updatestate_depend_eliminate]: 1.69128e-05
            [updatestate_assign_eliminate]: 3.59304e-06
            [updatestate_loads_eliminate]: 3.93391e-06
            [parameter_eliminate]: 1.67266e-06
            [a_2]: 6.25104e-05
            [accelerated_algorithm]: 6.13742e-06
            [auto_parall

![](docs/tutorial_b02.gif)